# KoBert
> "NLP"

- toc:true
- branch: master
- badges: true
- comments: true
- author: 전북대학교 통계학과 이강철
- categories: [python]
- hide :false
- published: true

 # GPU 용량확인 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


`런타임 초기화 하고 아래코드 돌려서 gpu용량 확인하고 코드돌리자....

In [3]:
import torch
import gc
gc.collect()

torch.cuda.empty_cache()

In [4]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=6345eee9a52764257055115c4c1bdb855bb6a977fb99d26598ba03998f44e708
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Gen RAM Free: 25.6 GB  |     Proc size: 1.2 GB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


# Colab 환경설정

!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

# github에서 KoBERT 파일을 로드 및 KoBERT모델 불러오기

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-pottzbzv/kobert-tokenizer_27f27d2645ba4a29802a219fa8635a27
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-pottzbzv/kobert-tokenizer_27f27d2645ba4a29802a219fa8635a27
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=5f5f6b0cf121b76b5513ae2e422e20a16c13aee407bc53785faa89c2d1fa0efc
  Stored in directory: /tmp/pip-ephem-wheel-cache-fzvxpa0y/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [6]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-cy1tez6g
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-cy1tez6g
     |████████████████████████████████| 132 kB 5.1 MB/s 
     |████████████████████████████████| 344 kB 44.6 MB/s 
     |████████████████████████████████| 47.3 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 52.9 MB/s 
     |████████████████████████████████| 1.2 MB 49.9 MB/s 
     |████████████████████████████████| 3.8 MB 39.5 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 6.5 MB 37.1 MB/s 
     |████████████████████████████████| 895 kB 58.7 MB/s 
     |████████████████████████████████| 596 kB 71.8 MB/s 
     |████████████████████████████████| 79 kB 10.1 MB/s 
     |████████████████████████████████| 8.6 MB 63.0 MB/s 
     |████████████████████████████████| 127 kB 78.9 MB/s 
  Created wheel for kobert: filename=ko

In [7]:
from kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab  = get_pytorch_kobert_model()

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


#  필요한 라이브러리 불러오기

In [8]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

# 데이터셋 불러오기

In [9]:
import pandas as pd
data = pd.read_csv('kobert입력데이터.csv')

In [10]:
data.head()

,document,topic,clean_txt1
0,1,15,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
1,2,5,올해 여자 작년 치매 고생 엄마 하늘 식구 엄마 우울증 무기력 상태 풀칠 강아지 연...
2,3,20,학생 고등학교 직전 학년 고등학교 선행 학습 학원 치열 주위 고등학교 국어 수학 과...
3,4,12,종국 동포 중소기업 금속 작업 교정 정리 표현 생략 어려움 세월 고국 관심 선거 고...
4,5,7,수고 위정자 헌법 수호 안전 위협 이슬람 영향 민족 지혜 문화 정책 내년 이슬람 관...


In [11]:
data["topic"] = data["topic"]-1

In [12]:
data.head()

,document,topic,clean_txt1
0,1,14,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
1,2,4,올해 여자 작년 치매 고생 엄마 하늘 식구 엄마 우울증 무기력 상태 풀칠 강아지 연...
2,3,19,학생 고등학교 직전 학년 고등학교 선행 학습 학원 치열 주위 고등학교 국어 수학 과...
3,4,11,종국 동포 중소기업 금속 작업 교정 정리 표현 생략 어려움 세월 고국 관심 선거 고...
4,5,6,수고 위정자 헌법 수호 안전 위협 이슬람 영향 민족 지혜 문화 정책 내년 이슬람 관...


In [13]:
data_list = []
for ques, label in zip(data['clean_txt1'], data['topic'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

#  입력 데이터셋을 토큰화하기

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

## 파라미터 셋팅

In [15]:
max_len = 200
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

# Train data & Test data





In [16]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [17]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

## 토큰화 및 패딩, 정수 인코딩 진행

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

# KoBERT 모델 구현

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=23,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.3).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-6},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/400 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.181016445159912 train acc 0.125
epoch 1 batch id 201 loss 2.133565664291382 train acc 0.3125
epoch 1 train acc 0.49095552884615384


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7405745967741936


  0%|          | 0/400 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.3983944058418274 train acc 0.875
epoch 5 batch id 201 loss 0.5450688004493713 train acc 0.9169776119402985
epoch 5 train acc 0.9228725961538461


  0%|          | 0/100 [00:00<?, ?it/s]

epoch 5 test acc 0.743366935483871


In [22]:
test_history

[0.7092842741935483,
 0.7161592741935483,
 0.725554435483871,
 0.7405745967741936,
 0.743366935483871]

In [23]:
loss_history

[array(3.1810164, dtype=float32),
 array(2.1335657, dtype=float32),
 array(1.1678196, dtype=float32),
 array(1.3618467, dtype=float32),
 array(0.8119287, dtype=float32),
 array(0.89912444, dtype=float32),
 array(0.6335163, dtype=float32),
 array(0.70157266, dtype=float32),
 array(0.3983944, dtype=float32),
 array(0.5450688, dtype=float32)]